# SOLID

Design principles intended to make object-oriented designs more **maintainable**.

These principles will help you to write clean code, with separation of concerns, cohesion, decoupling, helping with code reuse. They are:

* Single-Responsibility Principle (SRP)
* Open-Closed Principle (OCP)
* Liskov Substitution Principle (LSP)
* Interface Segregation Principle (ISP)
* Dependency Inversion Principle (DIP)

## Single-Responsibility Principle

`A class should have one, and only one, reason to change.` Robert C. Martin

This principle is applied to classes and functions, and they need to have one responsibility, a single purpose.

There are great benefits of applying SRP:

* **Code becomes easier to understand** - there is less happening when the implementation is more specific, as the classes are more concise.
* **Code becomes easier to maintain** - Less is more, as the classes don't deal with multiple things.
* **Code is changed less frequently** - in general, when requirements change, not every part of the application is going to change because the classes are more specific.
* **Code is testable** - because classes are specific and small, code can be thoroughly tested.

### Bad Example 

A classic example of how to violate the Single Responsibility Principle:


In [50]:
public class OrderProcessor
{
    private readonly Order _order;
    public OrderProcessor(Order order)
    {
        _order = order;
    }
    // Validate
    public void Validate()
    {
        //TODO: validate order
        Console.WriteLine($"validate order: {_order.OrderNumber}");
    }
    // Save
    public void Save()
    {
        //TODO: save order
        Console.WriteLine($"save order: {_order.OrderNumber}");
    }
    // Notify
    public void Notify()
    {
        //TODO: send notification
        Console.WriteLine($"send notification: {_order.OrderNumber}");
    }
}

Looking at the implementation above we can see the implementation of many responsibilities: `Validate()`, `Save()`, `Notify()`. This is a violation of SRP.

Problems we can identify with this implementation:

* **No cohesion** - There are different things in the same context of `OrderProcessor`.
* **Coupling** - Because of too many responsibilities, we have too many dependencies, leaving OrderProcessor in a fragile state in case changes are needed.
* **Testability** - It becomes more difficult to test the implementation.
* **Reusability** - Because the code is not modular, code reuse is affected.

### The right example

Based on the previous example, the `OrderProcessor` now becomes an **orchestrator**, and its responsibilities `Validate()`, `Save()`, `Notify()` are split into their own classes.

In [51]:
public class OrderProcessor
{
    private readonly OrderValidator _orderValidator;
    private readonly OrderSaver _orderSaver;
    private readonly OrderNotifier _orderNotifier;
    public OrderProcessor(OrderValidator orderValidator, OrderSaver orderSaver, OrderNotifier orderNotifier)
    {
        _orderValidator = orderValidator;
        _orderSaver = orderSaver;
        _orderNotifier = orderNotifier;
    }
    
    public void Process()
    {
        _orderValidator.Validate();
        _orderSaver.Save();
        _orderNotifier.Notify();
    }
}

Each class now has its own responsibility:

In [52]:
public class OrderValidator
{
    private readonly Order _order;
    public OrderValidator(Order order)
    {
        _order = order;
    }
    public void Validate()
    {
        //TODO: validate order
        Console.WriteLine($"validate order: {_order.OrderNumber}");
    }
}

In [53]:
public class OrderSaver
{
    private readonly Order _order;
    public OrderSaver(Order order)
    {
        _order = order;
    }
    public void Save()
    {
        //TODO: save order
        Console.WriteLine($"save order: {_order.OrderNumber}");
    }
}

In [54]:
public class OrderNotifier
{
    private readonly Order _order;
    public OrderNotifier(Order order)
    {
        _order = order;
    }
    public void Notify()
    {
        //TODO: send notification
        Console.WriteLine($"send notification: {_order.OrderNumber}");
    }
}

## Open-Closed Principle

`You should be able to extend a class behavior, without modifying it.` Robert C. Martin

This principle should be considered when requirements change, and code changes are required. Rather than modifying the implementation, you should be able to extend the implementation. A class should be open for extension but closed for modification.

There are great benefits of applying OCP:

* Avoid introducing code bugs - the chance of introducing bugs is very low, as the existing implementation is not modified.
* Dependent classes don't need to adapt - By not modifying existing implementations, there is no need for dependent classes to change.

### The wrong example

Let's take the example from SRP, in specific the `OrderSaver`. In this case a new requeriment is to be able to save the order in a Cosmos database. A violation of the Open-Closed Principle is shown below:

In [55]:
public class OrderSaver
{
    private readonly Order _order;
    public OrderSaver(Order order)
    {
        _order = order;
    }
    public void Save()
    {
        //TODO: save order
        Console.WriteLine($"save order to sql db: {_order.OrderNumber}");
    }
    public void SaveCosmos()
    {
        //TODO: save order to cosmos db
        Console.WriteLine($"save order to cosmos db: {_order.OrderNumber}");
    }
}

Looking at the implementation above we can see that the implementation for `OrderSaver` now has an extra method `SaveCosmos()`. 

We have directly modified an existing class, and for this method to be called, the dependent classes will need to adapt. 

Not only that, but we are also violating the Single Responsibility Principle (SRP)!

### The right example

When we are extending class behaviours, in **object-oriented programming (OOP)** we can use `inheritance`, `interfaces` or `composition`.

In my opinion, the use of interfaces is cleaner, as we dictate the use of a contract without any implementation.

Let's look at splitting concerns of `Save()` for saving Order to Sql, and `SaveCosmos()` for saving Order to Cosmos, using an interface `IOrderSaver`:

In [56]:
public interface IOrderSaver
{
    void Save();
}

In [57]:
public class SqlDbOrderSaver : IOrderSaver
{
    private readonly Order _order;
    public SqlDbOrderSaver(Order order)
    {
        _order = order;
    }
    public void Save()
    {
        //TODO: save order
        Console.WriteLine($"save order to sql db: {_order.OrderNumber}");
    }
}

In [58]:
public class CosmosDbOrderSaver : IOrderSaver
{
    private readonly Order _order;
    public CosmosDbOrderSaver(Order order)
    {
        _order = order;
    }
    public void Save()
    {
        //TODO: save order
        Console.WriteLine($"save order to cosmos db: {_order.OrderNumber}");
    }
}

This way the `OrderProcessor` doesn't need to change drastically, we should only replace `OrderSaver` with `IOrderSaver`:

In [59]:
public class OrderProcessor
{
    private readonly OrderValidator _orderValidator;
    private readonly IOrderSaver _orderSaver;
    private readonly OrderNotifier _orderNotifier;
    public OrderProcessor(OrderValidator orderValidator, IOrderSaver orderSaver, OrderNotifier orderNotifier)
    {
        _orderValidator = orderValidator;
        _orderSaver = orderSaver;
        _orderNotifier = orderNotifier;
    }
    
    public void Process()
    {
        _orderValidator.Validate();
        _orderSaver.Save();
        _orderNotifier.Notify();
    }
}

The Open-Closed principle is very useful, because it is very easy to add new behaviours with a very minimal chance of introducing bugs to existing implementations

## Liskov Substitution Principle

The Liskov Substitution Principle (LSP) is a principle that helps us to think about extending class behaviours, and this is applied in object-oriented programming (OOP) through inheritance, interfaces or composition.

`Derived classes must be substitutable for their base classes.` Robert C. Martin

Liskov Substitution Principle, written by Barbara Liskov in 1988, states the following: `"if S is a subtype of T, then objects of type T in a program may be replaced with objects of type S without altering any of the desirable properties of that program"`. In other words, any object of some class in an object-oriented program can be replaced by an object of a child class.

When applying this principle, inheritance and polymorphism should be carefully contextualized, otherwise, we can easily violate this principle.

### The wrong example

When abstractions are not well thought out, when systems increase in complexity and refactoring is not applied you may notice violations like that:

In [ ]:
public class Bird
{
    public string Name { get; }
    public Bird(string name)
    {
        Name = name;
    }
    public virtual void Eat()
    {
        Console.WriteLine($"{Name} eats.");
    }
    public virtual void LayEggs()
    {
        Console.WriteLine($"{Name} lays eggs.");
    }
    public virtual void Fly() // can all birds fly?
    {
        Console.WriteLine($"{Name} can fly.");
    }
}

Case 1: Because the abstraction wasn't well defined, exceptions are raised to justify an impossible action:

In [ ]:
public class Duck : Bird
{
    public Duck(string name) : base(name)
    {
    }
}
public class Kiwi : Bird
{
    public Kiwi(string name) : base(name)
    {
    }
    public override void Fly()
    {
        throw new Exception("Kiwi birds can't fly.");
    }
}

Case 2: Because it is legacy, some devs skipped the implementation by doing nothing:

In [ ]:
public class Penguin : Bird
{
    public Penguin(string name) : base(name)
    {
    }
    public override void Fly()
    {
        // Do nothing
    }
}

Looking at the implementations above, if we were to execute them in a `Program.cs`, unlike any other Principle violation that would result in bad but working code, this would lead to a buggy, difficult-to-maintain code.

### The right example

There are some ways to fix the previous implementations:

1. Leave the base class with only the minimal base behaviours, and leave the specialization in the derived class(es).

In [ ]:
public class Bird
{
    public string Name { get; }
    public Bird(string name)
    {
        Name = name;
    }
    public virtual void Eat()
    {
        Console.WriteLine($"{Name} eats.");
    }
    public virtual void LayEggs()
    {
        Console.WriteLine($"{Name} lays eggs.");
    }
}
public class Duck : Bird
{
    public Duck(string name) : base(name)
    {
    }
    public void Fly()
    {
        Console.WriteLine($"{Name} can fly.");
    }
}
public class Kiwi : Bird
{
    public Kiwi(string name) : base(name)
    {
    }
}

2. Use granular interfaces to dictate a contract of the implementation. By the way, this is related to the next principle Interface Segregation Principle (ISP).

In [ ]:
public interface IRegularBird
{
    string Name { get; }
    void Eat();
    void LayEggs();
}
public interface IFlyingBird
{
    string Name { get; }
    void Fly();
}
public class Bird : IRegularBird
{
    public string Name { get; }
    public Bird(string name)
    {
        Name = name;
    }
    public virtual void Eat()
    {
        Console.WriteLine($"{Name} eats.");
    }
    public virtual void LayEggs()
    {
        Console.WriteLine($"{Name} lays eggs.");
    }
}
public class FlyingBird : IRegularBird, IFlyingBird
{
    public string Name { get; }
    public FlyingBird(string name)
    {
        Name = name;
    }
    public virtual void Eat()
    {
        Console.WriteLine($"{Name} eats.");
    }
    public virtual void LayEggs()
    {
        Console.WriteLine($"{Name} lays eggs.");
    }
    public virtual void Fly()
    {
        Console.WriteLine($"{Name} can fly.");
    }
}
public class Kiwi : Bird
{
    public Kiwi(string name) : base(name)
    {
    }
}
public class Duck : FlyingBird
{
    public Duck(string name) : base(name)
    {
    }
}

## Interface Segregation Principle

`Make fine grained interfaces that are client specific.` Robert C. Martin

The Interface Segregation Principle (ISP) is a principle that helps us to think about creating many specialized interfaces instead of a single general-purpose interface.

When interfaces aggregate lots of generic behaviours they become difficult to maintain and evolve. The rule of thumb is `"the more specific the better"``.

If you think about the Single-Responsibility Principle (SRP), the same principles are applied here.

### The wrong example

The same example as SRP, but this time we represent it through the use of an interface, a fat one:

In [60]:
public interface IOrderProcessor
{
    void Validate();
    void Save();
    void Notify();
}

In this case, we only want to validate an order, but because we have a single general-purpose interface, the client is mandated to implement methods that aren't needed:

In [ ]:
public class OrderValidator : IOrderProcessor
{
    private readonly Order _order;
    public OrderValidator(Order order)
    {
        _order = order;
    }
    // Validate
    public void Validate()
    {
        //TODO: validate order
        Console.WriteLine($"validate order: {_order.OrderNumber}");
    }
    // Save
    public void Save()
    {
        throw new NotImplementedException();
    }
    // Notify
    public void Notify()
    {
        throw new NotImplementedException();
    }
}

In general, you get to see `NotImplementedException` when the principle is violated.

### The right example

The more specific the better:

In [ ]:
public interface IOrderValidator
{
    void Validate();
}

Similar to the right example from SRP, this one uses the specific interface `IOrderSaver`:

In [ ]:
public class OrderSaver : IOrderSaver
{
    private readonly Order _order;
    public OrderSaver(Order order)
    {
        _order = order;
    }
    public void Save()
    {
        //TODO: save order
        Console.WriteLine($"save order: {_order.OrderNumber}");
    }
}

## Dependency Inversion Principle

`Depend on abstractions, not on concretions.` Robert C. Martin

The principle also mentions that:

* High-level modules should not depend upon low-level modules, both should depend upon abstractions.
* Abstractions should not depend upon details. Details should depend upon abstractions.

We can associate this principle with decoupling because the idea is to disassociate module dependencies. This has great benefits:

* Resiliency - things are not going to easily break, minimizing the impact of changes.
* Reusability - because of the dependency on abstractions, the code becomes much more flexible and reusable.
* Testability - decoupling makes it easier to unit test.

Abstraction can be compared to a process of hiding the implementation details from the client but making the functionality available to this client. In other words, the client has the information on what the object does instead of how it does it.

Interfaces - Since all the methods of an interface are abstract and the client doesn't know how a method is written (except the method signature/prototype), abstraction is achieved when using interfaces. Interfaces also promote decoupling, as we have already seen it in (OCP)1 and (ISP)2.

### The wrong example

The same example as (SRP), which uses concretions in constructors:

In [ ]:
public class OrderProcessor
{
    private readonly OrderValidator _orderValidator;
    private readonly OrderSaver _orderSaver;
    private readonly OrderNotifier _orderNotifier;
    public OrderProcessor(OrderValidator orderValidator, OrderSaver orderSaver, OrderNotifier orderNotifier) // should we use concretions?
    {
        _orderValidator = orderValidator;
        _orderSaver = orderSaver;
        _orderNotifier = orderNotifier;
    }
    
    public void Process()
    {
        _orderValidator.Validate();
        _orderSaver.Save();
        _orderNotifier.Notify();
    }
}


### The right example

Depend on abstractions, not on concretions.


In [62]:
public interface IOrderValidator
{
    void Validate();
}

In [64]:
public interface IOrderSaver
{
    void Save();
}

In [65]:
public interface IOrderNotifier
{
    void Notify();
}

In [66]:

public class OrderProcessor
{
    private readonly IOrderValidator _orderValidator;
    private readonly IOrderSaver _orderSaver;
    private readonly IOrderNotifier _orderNotifier;
    public OrderProcessor(IOrderValidator orderValidator, IOrderSaver orderSaver, IOrderNotifier orderNotifier)
    {
        _orderValidator = orderValidator;
        _orderSaver = orderSaver;
        _orderNotifier = orderNotifier;
    }
    
    public void Process()
    {
        _orderValidator.Validate();
        _orderSaver.Save();
        _orderNotifier.Notify();
    }
}



By using the interfaces, the high-level `OrderProcessor` is not depending on low-level modules, but on abstractions represented by the contracts `IOrderValidator`, `IOrderSaver` and `IOrderNotifier`.